In [2]:
%load_ext autoreload
import sys
if not '/Users/nlarusstone/Documents/MPhil/Research/cf_fba' in sys.path:
    sys.path.append('/Users/nlarusstone/Documents/MPhil/Research/cf_fba')
import src.utils as utils
%autoreload 2

In [3]:
import cobra
import cobra.test
import pandas as pd
import numpy as np
import difflib
import re
import Bio.PDB.Polypeptide

In [90]:
import Bio.PDB.Polypeptide as pdb
import Bio.SeqUtils as su

aa_mix = pd.read_csv('../data/aa_mix.csv', index_col='AA')
nrg_mix = pd.read_csv('../data/energy_mix.csv', index_col='compound')
with open('../genes/rfp.txt', 'r') as f:
    seq = f.read()
cfps_conc = pd.read_csv('../data/cfps_start.csv', index_col='compound')

# amt in g, vol in mL, mw in g/mol
def calc_conc(amt, vol, mw=None, seq=None, seq_type=None):
    # seq can be DNA or protein or an amino acid
    if seq:
        mw = su.molecular_weight(seq, seq_type)
    elif not mw:
        raise Exception('Need a molecular weight for non-DNA')
    conc = (amt * 1000) / (vol * mw)
    # returns Molar concentrations
    return conc

def conc_dilution(start_conc, vol_add, tot_vol):
    return start_conc * (vol_add / tot_vol)

aa_mix['start_conc'] = aa_mix.apply(lambda row: calc_conc(row['weight_add'], 1, 
                                                          seq=pdb.three_to_one(row.name.upper()), seq_type='protein'), axis=1)
aa_mix['conc_add'] = conc_dilution(aa_mix['start_conc'], aa_mix['vol_add'], aa_mix['vol_add'].sum())
#pi
pi_conc = calc_conc(0.15, 5, mw=611.77)
nrg_mix['start_conc'] = nrg_mix.apply(lambda row: calc_conc(row['amt'], row['fill'], mw=row['mw']), axis=1)
nrg_mix['conc_add'] = conc_dilution(nrg_mix['start_conc'], nrg_mix['vol_add'], nrg_mix['vol_add'].sum())
dna_conc = calc_conc(0.000750, 0.00496, seq=seq, seq_type='DNA')

for cmpnd, vals in nrg_mix.iterrows():
    cfps_conc.loc[cmpnd] = [vals['conc_add'], 5]
for aa, vals in aa_mix.iterrows():
    cfps_conc.loc[aa] = [vals['conc_add'], 10]
cfps_conc.loc['GENE'] = [dna_conc, 4.96]
cfps_conc['final_conc'] = conc_dilution(cfps_conc['start_conc'], cfps_conc['amt'], 50)

In [4]:
model = cobra.io.read_sbml_model(filename='../models/iJO1366.xml')

In [57]:
varner = cobra.io.load_json_model('../models/varner.json')
def extract_txtl_rxns(model):
    aa_metabs = []
    for aa in Bio.PDB.Polypeptide.aa3:
        aa_metabs += model.metabolites.query(aa.lower())
    aa_rxns = []
    for aa_metab in aa_metabs:
        aa_rxns += aa_metab.reactions
    mrna_rxns = model.reactions.query(re.compile('mrna', re.IGNORECASE))
    trna_rxns = model.reactions.query('tRNA_c')
    tx_rxns = model.reactions.query('transcription')
    tl_rxns = model.reactions.query('translation')
    prot_rxns = model.reactions.query('PROTEIN')
    #txtl_rxns = list(set(aa_rxns).union(tx_rxns).union(tl_rxns).union(prot_rxns).union(mrna_rxns))
    txtl_rxns = list(set(tx_rxns).union(tl_rxns).union(prot_rxns).union(mrna_rxns).union(trna_rxns))
    return txtl_rxns
txtl_rxns = extract_txtl_rxns(varner)
txtl_rxns[:10]

[<Reaction tRNA_c_charging_M_lys_L_c_RFP at 0x1116e7410>,
 <Reaction transcription_RFP at 0x1116e7050>,
 <Reaction tRNA_c_charging_M_pro_L_c_RFP at 0x1116e74d0>,
 <Reaction tRNA_c_charging_M_val_L_c_RFP at 0x1116e7610>,
 <Reaction mRNA_degradation_RFP at 0x1116e7090>,
 <Reaction translation_initiation_RFP at 0x1116e70d0>,
 <Reaction tRNA_c_charging_M_gln_L_c_RFP at 0x1116e72d0>,
 <Reaction translation_RFP at 0x1116e7110>,
 <Reaction tRNA_c_exchange_reverse at 0x1116e7690>,
 <Reaction tRNA_c_charging_M_ala_L_c_RFP at 0x1116e7150>]

In [11]:
metab_ids = [m.id.rsplit('_c', 1)[0] for m in model.metabolites if m.compartment == 'c']
varner_to_ijo = {'5pbdra': 'pram',
                'faicar': 'fprica',
                '78dhf': 'dhpt',
                '78mdp': '6hmhptpp',
                'cadav': '15dap',
                'cair': '5aizc',
                'clasp': 'cbp',
                'fgar': 'fgam',
                'gaba': '4abut',
                'glycoA': 'gcald',
                'nh3': 'nh4',
                'he': 'h',
                'omp': 'orot5p',
                 'or': 'orot',
                 'prop': 'ppcoa',
                 'mglx': 'mthgxl',
                 'saicar': '25aics',
                 '4adochor': '4adcho'
                }
metab_ids[:10]

['10fthf',
 '12dgr120',
 '12dgr140',
 '12dgr141',
 '12dgr160',
 '12dgr161',
 '12dgr180',
 '12dgr181',
 '12ppd__R',
 '12ppd__S']

In [10]:
def varner_to_cobra(model, metab, metab_ids, varner_to_ijo):
    if metab.id.startswith('M_'):
        metab_stem = metab.id.split('M_')[1].rsplit('_c', 1)[0]
        #print metab_stem
        if 'tRNA' in metab_stem:
            aa = metab_stem.split('_', 1)[0]
            metab_name = aa + 'trna'
        elif not metab_stem in metab_ids:
            #query = varner_to_ijo[metab_stem]
            #print metab_stem
            if metab_stem in varner_to_ijo:
                #print 'matched'
                metab_name = varner_to_ijo[metab_stem]
            elif '_L' in metab_stem or '_D' in metab_stem:
                #print difflib.get_close_matches(metab_stem, metab_ids, 1, 0.7)
                metab_name = difflib.get_close_matches(metab_stem, metab_ids, 1, 0.7)[0]
            else:
                print 'TODO: ', metab_stem
                raise Exception
        else:
            metab_name = metab_stem
    else:
        try:
            model.metabolites.get_by_id(metab_name)
        except:
            model.metabolites.add(metab)
    return model.metabolites.get_by_id(metab_name + '_c')

In [68]:
def add_txtl(model, txtl_rxns):
    mod = model.copy()
    for rxn in txtl_rxns:
        #print rxn
        for metab, amt in rxn.metabolites.items():
            if not metab.id.startswith('M_'):
                #print 'EXCEPT:', metab
                continue
            new_metab = varner_to_cobra(mod, metab, metab_ids, varner_to_ijo)
            rxn.add_metabolites({metab: -1 * amt})
            rxn.add_metabolites({new_metab: amt})
        mod.add_reaction(rxn)
    return mod
model_txtl = add_txtl(model, txtl_rxns)

In [74]:
def get_aa_metab(model, aa, cmpt='c'):
    return model.metabolites.query('{0}__._{1}'.format(aa, cmpt))

In [75]:
def replace_metab(mod, metab):
    new_id = re.sub(r'_.$', '_c', metab.id)
    try:
        cyt = mod.metabolites.get_by_id(new_id)
    except:
        cyt = metab
        cyt.id = new_id
        cyt.compartment = 'c'
    return cyt

In [70]:
def coalesce_cmpts(model):
    mod = model.copy()
    for rxn in mod.reactions:
        if 'p' in rxn.compartments or 'e' in rxn.compartments:
            #mod.remove_reactions(reactions=[rxn])
            for metab, amt in rxn.metabolites.items():
                cyt = replace_metab(mod, metab)
                rxn.add_metabolites({metab: -1 * amt})
                rxn.add_metabolites({cyt: amt})
            rxn.comparments = set('c')
            #mod.add_reaction(reaction=rxn)
    for m in mod.metabolites.query(r'.*_e$'):
        assert(len(m.reactions) == 0)
        m.remove_from_model(destructive=True)
    for m in mod.metabolites.query(r'.*_p$'):
        assert(len(m.reactions) == 0)
        m.remove_from_model(destructive=True)
    return mod
model_cyt = coalesce_cmpts(model)
model_cyt_txtl = coalesce_cmpts(model_txtl)

In [76]:
def strip_exchanges(mod, reactants):
    # Delete transmembrane transport reactions
    model = mod.copy()

    # Add exchange reactions for all metabolites
    # Exchange‐reactions for non‐substrate metabolites have lb of 0 and ub of 1000 by default
    #all_metab_ex = utils.gen_metab_dict(model, model.metabolites, cnvt=False)

    #utils.add_ex(model, all_metab_ex)
    exs = set()
    for metab in reactants:
        if metab == 'trna':
            for trna in model.metabolites.query('trna'):
                exs = exs.union(trna.reactions.intersection(model.exchanges))
        elif metab.upper() in pdb.aa3:
            aas = get_aa_metab(model, metab.lower(), cmpt='c')
            for aa in aas:
                exs = exs.union(aa.reactions.intersection(model.exchanges))
        else:
            m = model.metabolites.get_by_id('{0}_c'.format(metab))
            exs = exs.union(m.reactions.intersection(model.exchanges))
    model.remove_reactions(exs)
    #['EX_glc_e', 'EX_pi_e', 'EX_mg2_e', 'EX_k_e', 'EX_nh4_e'])

    # As objective function, we selected the exchange reaction which corresponds to the target metabolite 
    # for which a pathway should be determined.   
    return model
model_bare = strip_exchanges(model_cyt, cfps_conc.index[:-1])
model_bare_txtl = strip_exchanges(model_cyt_txtl, cfps_conc.index[:-1])

In [77]:
def build_medium(model, cfps_conc):
    mod = model.copy()
    
    for metab, vals in cfps_conc.iterrows():
        flux = utils.conc_to_flux(vals['final_conc']) * 100

        if metab == 'trna':
            ms = model.metabolites.query('trna')
        elif metab.upper() in pdb.aa3:
            ms = get_aa_metab(model, metab.lower(), cmpt='c')
        else:
            ms = mod.metabolites.query(r'^{0}_c'.format(metab))
        for m in ms:
            rxn_nm = 'EX_' + m.id
            mod.add_boundary(metabolite=m, type='exchange', lb=0, ub=flux)
            #mod.add_boundary(metabolite=m, type='cfps-medium', reaction_id=rxn_nm, lb=0, ub=flux) 
    return mod
model_cf = build_medium(model_bare, cfps_conc)
model_cf_txtl = build_medium(model_bare_txtl, cfps_conc)

In [81]:
model_cf_txtl_rfp = model_cf_txtl.copy()
model_cf_txtl_rfp.objective = model_cf_txtl_rfp.reactions.PROTEIN_export_RFP
model_cf_txtl_comb = model_cf_txtl.copy()
model_cf_txtl_comb.objective = {model_cf_txtl_comb.reactions.BIOMASS_Ec_iJO1366_core_53p95M: 1, model_cf_txtl_comb.reactions.PROTEIN_export_RFP: 100}

In [80]:
sol = model_cf.optimize()
model_cf.summary()

IN FLUXES            OUT FLUXES            OBJECTIVES
-------------------  --------------------  ---------------------
h_c        7.51      h2o_c      18.2       BIOMASS_Ec_i...  2.55
o2_c       3.36      nh4_c       8.34
pser__L_c  1.5       co2_c       7.31
ser__L_c   1.5       h2s_c       2.73
trp__L_c   1.49      h2o2_c      2.69
asn__L_c   1.48      akg_c       2.36
athr__L_c  1.48      indole_c    1.34
thr__L_c   1.48      ac_c        0.0993
glu__D_c   1.48      nac_c       0.0131
pro__L_c   1.48      hcys__L_c   0.00114
asp__L_c   1.48      mththf_c    0.00114
ala__D_c   1.48      5drib_c     0.000574
ala__L_c   1.48      4crsol_c    0.000569
gln__L_c   1.48      trnaglu_c   0.000434
cys__D_c   1.48      meoh_c      5.1e-06
cys__L_c   1.48      amob_c      5.1e-06
leu__L_c   1.15
val__L_c   1.08
lys__L_c   0.875
arg__L_c   0.754
ile__L_c   0.741
k_c        0.498
glu__L_c   0.483
phe__L_c   0.473
met__L_c   0.394
tyr__L_c   0.352
ser__D_c   0.318
his__L_c   0.242
glc__D_c   0.226

In [82]:
sol = model_cf_txtl_rfp.optimize()
model_cf_txtl_rfp.summary()

IN FLUXES             OUT FLUXES              OBJECTIVES
--------------------  ----------------------  -----------------------
h_c        14.2       h2o_c        13.2       PROTEIN_expo...  0.0124
arg__L_c    1.79      nh4_c         8.65
o2_c        1.72      co2_c         3.74
pser__L_c   1.5       succ_c        3.74
ser__L_c    1.5       for_c         2.8
trp__L_c    1.49      h2o2_c        1.71
asn__L_c    1.48      indole_c      1.31
athr__L_c   1.48      h2s_c         1.25
thr__L_c    1.48      urea_c        0.98
glu__D_c    1.48      nac_c         0.0134
glu__L_c    1.48      PROTEIN_RFP   0.0124
pro__L_c    1.48      hcys__L_c     0.00108
asp__L_c    1.48      mththf_c      0.00108
ala__D_c    1.48      5drib_c       0.000544
ala__L_c    1.48      4crsol_c      0.000539
gln__L_c    1.48      trnaglu_c     0.000434
cys__L_c    1.48      meoh_c        4.84e-06
leu__L_c    1.26      amob_c        4.84e-06
val__L_c    1.25
lys__L_c    1.1
ile__L_c    0.826
phe__L_c    0.609
tyr__L_c

In [83]:
sol = model_cf_txtl_comb.optimize()
model_cf_txtl_comb.summary()

IN FLUXES             OUT FLUXES              OBJECTIVES
--------------------  ----------------------  ---------------------
h_c        17         h2o_c        16.4       BIOMASS_Ec_i...  2.55
arg__L_c    1.79      nh4_c         7.86      PROTEIN_expo...  1.24
o2_c        1.64      co2_c         5.27
pser__L_c   1.5       succ_c        3.16
ser__L_c    1.5       h2o2_c        1.64
trp__L_c    1.49      indole_c      1.3
asn__L_c    1.48      h2s_c         1.23
athr__L_c   1.48      urea_c        0.941
thr__L_c    1.48      for_c         0.407
glu__D_c    1.48      nac_c         0.0131
glu__L_c    1.48      PROTEIN_RFP   0.0124
pro__L_c    1.48      hcys__L_c     0.00114
asp__L_c    1.48      mththf_c      0.00114
ala__D_c    1.48      5drib_c       0.000574
ala__L_c    1.48      4crsol_c      0.000569
gln__L_c    1.48      trnaglu_c     0.000434
cys__L_c    1.48      meoh_c        5.1e-06
leu__L_c    1.32      amob_c        5.1e-06
val__L_c    1.3
lys__L_c    1.15
ile__L_c    0.865
phe

In [84]:
cobra.io.write_sbml_model(filename='../models/ecoli_cf_base.sbml', cobra_model=model_cf)
cobra.io.write_sbml_model(filename='../models/ecoli_cf_txtl_rfp_base.sbml', cobra_model=model_cf_txtl_rfp)
cobra.io.write_sbml_model(filename='../models/ecoli_cf_txtl_comb_base.sbml', cobra_model=model_cf_txtl_comb)

In [ ]:
#m.metabolites.mRNA_RFP.elements = {'C': 6475, 'H': 8586, 'N': 2552, 'O': 5406, 'P': 678}
#m.reactions.transcription_RFP.annotation = {'SBO': 'SBO:0000631'}
def validate_model(m):
    cobra.manipulation.validate.check_reaction_bounds(model=m)
    cobra.manipulation.validate.check_metabolite_compartment_formula(model=m)
    cobra.manipulation.validate.check_mass_balance(model=m)

In [24]:
df = pd.read_csv('../data/Karim_MetEng_2018_Figure2_Data.csv')
df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)
df.head()

,AVG,STD,AVG.1,STD.1,Mg(Glu)2,NH4(Glu),K(Glu),Glucose,K2HPO4,NAD,ATP,CoA
0,0.00,0.0,0.00,0.00,8.0,10.0,134,0,10,0.5,0.0,0.5
1,0.00,0.0,0.00,0.00,8.0,10.0,134,200,10,0.0,0.0,0.5
2,0.00,0.0,0.00,0.00,8.0,10.0,134,200,10,0.5,0.0,2.0
3,0.00,0.0,0.00,0.00,8.0,10.0,134,200,10,0.5,0.0,1.6
4,1.21,1.7,0.09,0.13,8.0,10.0,134,200,10,0.5,2.0,0.5


In [55]:
reduced = df.loc[:, ['Glucose', 'NH4(Glu)', 'NAD', 'CoA']].drop_duplicates(keep='first')
reduced['AVG.1'] = df.loc[reduced.index, 'AVG.1']
reduced.to_csv('../data/reduced.csv')

In [24]:
utils.change_obj(model=model_cf_med, metab=model_cf_med.metabolites.btol_c)
utils.change_obj(model=model_cf, metab=model_cf.metabolites.btol_c)

In [66]:
def add_addl_reactants(model, df):
    mod = model.copy()
    addl_reagent_nms = ['mg2', 'nh4', 'k', 'glc__D', 'pi', 'nad', 'atp', 'coa']
    objs = []
    for i, row in df.iterrows():
        with mod:
            metab_dict = utils.gen_metab_dict(mod, addl_reagent_nms, map(lambda x: -1 * utils.conc_to_flux(x), row[4:]))
            rxn = utils.add_exchange(mod, metab_dict, additive=True)
            #different_mediums(mod, model)
            #mod.add_reactions(reaction_list=sol[0])
            obj = mod.slim_optimize()
            objs.append(obj)
            #print 'Obj: {0}'.format(obj.objective_value)
    return objs

In [70]:
model.metabolites.mg2_c

Metabolite identifier,mg2_c
Name,magnesium
Memory address,0x0109757410
Formula,Mg
Compartment,c
In 5 reaction(s),"Ec_biomass_iJO1366_core_53p95M, MG2t3_2pp, MG2uabcpp, Ec_biomass_iJO1366_WT_53p95M, MG2tpp"


In [69]:
utils.add_addl_reactants(model_cf_med, df)

[1.0632867132867072,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 2.895833333333339,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 2.340277777777785,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.7847222222222205,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 0.11805555555555675,
 0.6736111111111152,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435]

In [12]:
cobra.io.save_json_model(filename='Full_Ecoli_CF_Base.json', model=model_cf_med)

In [81]:
model.medium

{'EX_ca2_e': 1000.0,
 'EX_cbl1_e': 0.01,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [13]:
model.optimize()
model.summary()

IN FLUXES             OUT FLUXES           OBJECTIVES
--------------------  -------------------  ----------------------
o2_e       17.6       h2o_e     45.6       Ec_biomass_i...  0.982
nh4_e      10.6       co2_e     19.7
glc__D_e   10         h_e        9.03
pi_e        0.948     mththf_c   0.00044
so4_e       0.248     5drib_c    0.000221
k_e         0.192     4crsol_c   0.000219
fe2_e       0.0158    amob_c     1.96e-06
mg2_e       0.00852   meoh_e     1.96e-06
ca2_e       0.00511
cl_e        0.00511
cu2_e       0.000697
mn2_e       0.000679
zn2_e       0.000335
ni2_e       0.000317
mobd_e      0.000127
cobalt2_e   2.46e-05


In [17]:
fl = -1 * utils.conc_to_flux(200)
print fl
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = fl, fl
model.optimize()
model.summary()

-83.3333333333
IN FLUXES              OUT FLUXES            OBJECTIVES
---------------------  --------------------  ---------------------
o2_e       142         h2o_e     379         Ec_biomass_i...  8.28
nh4_e       89.5       co2_e     160
glc__D_e    83.3       h_e        76.1
pi_e         7.99      mththf_c    0.00371
so4_e        2.09      5drib_c     0.00186
k_e          1.62      4crsol_c    0.00185
fe2_e        0.133     amob_c      1.66e-05
mg2_e        0.0718    meoh_e      1.66e-05
ca2_e        0.0431
cl_e         0.0431
cu2_e        0.00587
mn2_e        0.00572
zn2_e        0.00282
ni2_e        0.00268
mobd_e       0.00107
cobalt2_e    0.000207


In [18]:
fl = -1 * utils.conc_to_flux(120)
print fl
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = fl, fl
model.optimize()
model.summary()

-50.0
IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  ---------------------
o2_e       85.6       h2o_e     227         Ec_biomass_i...  4.96
nh4_e      53.6       co2_e      96.2
glc__D_e   50         h_e        45.6
pi_e        4.79      mththf_c    0.00222
so4_e       1.25      5drib_c     0.00112
k_e         0.969     4crsol_c    0.00111
fe2_e       0.0797    amob_c      9.93e-06
mg2_e       0.0431    meoh_e      9.93e-06
ca2_e       0.0258
cl_e        0.0258
cu2_e       0.00352
mn2_e       0.00343
zn2_e       0.00169
ni2_e       0.0016
mobd_e      0.00064
cobalt2_e   0.000124


In [92]:
utils.change_obj(model, model.metabolites.btol_c)

In [20]:
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = -10, 1000
model.reactions.EX_glc_e
model.optimize()
model.summary()

IN FLUXES        OUT FLUXES     OBJECTIVES
---------------  -------------  ---------------
nh4_e     12.5   h2o_e   47.5   DM_btol_c  1.79
glc__D_e  10     co2_e   15.2
o2_e       9.87  h_e      5.37
pi_e       5.37  btol_c   1.79
so4_e      1.79


In [21]:
fl = -1 * utils.conc_to_flux(200)
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = fl, fl
model.optimize()
model.summary()

IN FLUXES        OUT FLUXES     OBJECTIVES
---------------  -------------  ---------------
nh4_e     106    h2o_e   394    DM_btol_c  15.1
glc__D_e   83.3  co2_e   123
o2_e       77.7  h_e      45.3
pi_e       45.3  btol_c   15.1
so4_e      15.1


In [22]:
fl = -1 * utils.conc_to_flux(120)
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = fl, fl
model.optimize()
model.summary()

IN FLUXES        OUT FLUXES      OBJECTIVES
---------------  --------------  ---------------
nh4_e     63.3   h2o_e   237     DM_btol_c  9.04
glc__D_e  50     co2_e    74
o2_e      46.8   h_e      27.1
pi_e      27.1   btol_c    9.04
so4_e      9.04


In [24]:
fl = -1 * utils.conc_to_flux(200)
fl_nh4 = -1 * utils.conc_to_flux(10)
print fl, fl_nh4
glc = model.reactions.EX_glc_e
nh4 = model.reactions.EX_nh4_e
glc.lower_bound, glc.upper_bound = fl, fl
nh4.lower_bound, nh4.upper_bound = fl_nh4, fl_nh4
model.optimize()
model.summary()

-83.3333333333 -4.16666666667
IN FLUXES         OUT FLUXES         OBJECTIVES
----------------  -----------------  ----------------
glc__D_e  83.3    h2o_e     122      DM_btol_c  0.595
o2_e      61.1    co2_e     101
nh4_e      4.17   h_e        61.7
pi_e       1.79   3hpp_e     28.8
so4_e      0.595  mththf_c   28.8
                  hxa_e      21.6
                  akg_e       4.76
                  btol_c      0.595


In [97]:
fl = -1 * utils.conc_to_flux(200)
fl_nh4 = -1 * utils.conc_to_flux(10)
fl_pi = -1 * utils.conc_to_flux(10)
fl_mg = -1 * utils.conc_to_flux(8)
print fl, fl_nh4
glc = model.reactions.EX_glc_e
nh4 = model.reactions.EX_nh4_e
pi = model.reactions.EX_pi_e
mg = model.reactions.EX_mg2_e
def update_bnds(rxn, fl): 
    rxn.lower_bound, rxn.upper_bound = fl, 0 
glc.lower_bound, glc.upper_bound = fl, fl
nh4.lower_bound, nh4.upper_bound = fl_nh4, fl_nh4
update_bnds(pi, fl_pi)
update_bnds(mg, fl_mg)
model.optimize()
model.summary()

-83.3333333333 -4.16666666667
IN FLUXES         OUT FLUXES           OBJECTIVES
----------------  -------------------  ----------------
glc__D_e  83.3    dha_e       73.1     DM_btol_c  0.595
o2_e      13.6    co2_e       53.2
nh4_e      4.17   12ppd__R_e  42.7
pi_e       1.83   h_e         40.3
so4_e      0.595  ac_e        28.8
                  h2o_e       25.4
                  mal__L_e     4.15
                  glyclt_e     1.48
                  mththf_c     1.48
                  btol_c       0.595
                  glyc3p_e     0.0425


In [103]:
elts = ['mg2', 'nh4', 'k', 'glc', 'pi', 'nadh', 'atp', 'coa']
row = df.iloc[36, 4:]
fluxes = [-1 * utils.conc_to_flux(row[i]) for i in range(len(elts))]
print elts
print row.tolist()
for elt, flux in zip(elts, fluxes):
    #print elt, '{0}_c'.format(elt)#, model.metabolites.get_by_id('{0}_c'.format(elt))
    try:
        rxn = model.reactions.get_by_id('EX_{0}_e'.format(elt))
    except KeyError:
        break
        #model.add_boundary(reaction_id='EX_{0}_e'.format(elt), type='Exchange', 
        #                   metabolite=model.metabolites.get_by_id('{0}_c'.format(elt)), ub=flux)
        #rxn = model.reactions.get_by_id('EX_{0}_e'.format(elt))
    rxn.lower_bound, rxn.upper_bound = flux, 0
model.optimize()
model.summary()

['mg2', 'nh4', 'k', 'glc', 'pi', 'nadh', 'atp', 'coa']
[8.0, 10.0, 134.0, 200.0, 4.0, 0.5, 0.0, 0.5]
IN FLUXES        OUT FLUXES        OBJECTIVES
---------------  ----------------  ----------------
o2_e      6.5    h2o_e     18.6    DM_btol_c  0.556
glc__D_e  4.49   co2_e      7.75
nh4_e     3.89   fe3_e      1.67
fe2_e     1.67   mththf_c   1.06
pi_e      1.67   btol_c     0.556
so4_e     0.556


In [22]:
df2 = pd.read_csv(filepath_or_buffer='../data/4_5_18_T7_C6N_mRFP.TXT', skiprows=6)

In [27]:
df2.columns

Index([u'Well', u'Unnamed: 1', u'C01', u'C02', u'C03', u'C04', u'C05', u'C06',
       u'C07', u'C08', u'C09', u'D01', u'D02', u'D03', u'D04', u'D05', u'D06',
       u'D07', u'D08', u'D09', u'E01', u'E02', u'E03', u'E04', u'E05', u'E06',
       u'E07', u'E08', u'E09', u'F01', u'F02', u'F03', u'F04', u'F05', u'F06',
       u'F07', u'F08', u'F09', u'F10'],
      dtype='object')